# 전처리 후  저장

In [1]:
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt
from augment import augment
import os
from keras.datasets.cifar100 import load_data
from keras.utils import to_categorical



Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test)=tf.keras.datasets.cifar100.load_data(label_mode='fine')

In [3]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_train[0][1].shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)
(32, 3)


In [4]:
def normalize(X_train, X_test):

    mean = np.mean(X_train, axis=(0, 1, 2, 3))
    std = np.std(X_train, axis=(0, 1, 2, 3))

    X_train = (X_train - mean) / std
    X_test = (X_test - mean) / std

    return X_train, X_test

In [5]:
x_train, x_test = normalize(x_train, x_test)
# y_train = to_categorical(y_train, 100)
# y_test = to_categorical(y_test, 100)
seed = 777
np.random.seed(seed)
np.random.shuffle(x_train)
np.random.seed(seed)
np.random.shuffle(y_train)

# CNN modeling

In [6]:
learning_rate = 0.001
training_epoch = 10
minibatch_size = 100

In [7]:
training = tf.placeholder(tf.bool)
X = tf.placeholder(tf.float32 , shape = [None, 32,32,3 ])
Y = tf.placeholder(tf.int32, shape = [None, 100])
X_img = tf.reshape(X, [-1, 32, 32, 1])

conv1 = tf.layers.conv2d(inputs = X_img, filters = 32, kernel_size = [3,3], padding='SAME', activation = tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size=[2,2], padding='SAME', strides = 2)
dropout1 = tf.layers.dropout(inputs = pool1, rate = 0.7, training = training)
#after shape(100, 150)

conv2 = tf.layers.conv2d(inputs = dropout1, filters = 64, kernel_size=[3,3], padding='SAME',activation = tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size=[2,2], padding='SAME', strides=2)
dropout2 = tf.layers.dropout(inputs = pool2, rate=0.7, training = training)
#after shape(50, 75)

conv3 = tf.layers.conv2d(inputs = dropout2, filters = 128, kernel_size=[3,3], padding = 'SAME', activation = tf.nn.relu)
pool3 = tf.layers.max_pooling2d(inputs = conv3, pool_size = [2,2], padding='SAME', strides = 2)
dropout3 = tf.layers.dropout(inputs = pool3, rate = 0.5, training=training)
#after shape(25, 38)

flat = tf.reshape(dropout3 , [-1, 32*32*128])
dense4 = tf.layers.dense(inputs = flat, units = 256, activation = tf.nn.relu)
dropout4 = tf.layers.dropout(inputs = dense4, rate=0.5, training = training)

logits = tf.layers.dense(inputs = dropout4, units = 4)
hypothesis = tf.nn.softmax(logits)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

n_mb = 50000//minibatch_size

for i in range(training_epoch):
    for j in range(n_mb):
        _, c = sess.run([optimizer, cost], feed_dict={X:x_train[j*minibatch_size : (j+1)*minibatch_size], 
                                                      Y: y_train[j*minibatch_size: (j+1)*minibatch_size],
                                                      training:True})
    _, c = sess.run([optimizer, cost], feed_dict = {X: x_test[n_mb * minibatch_size : ], 
                                                    Y: y_train[n_mb * minibatch_size : ],
                                                    training:True })
    print('cost : ',c)
#     if i%1 ==0:
#         print('cost: ',c)

ValueError: Cannot feed value of shape (100, 1) for Tensor 'Placeholder_2:0', which has shape '(?, 100)'